In [1]:
import json

In [2]:
j = '''
{
    "a": 100,
    "b": [1,2,3],
    "c": "python",
    "d": {
    "e": 4,
    "f": 5.5
    }
}'''

In [3]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        print("decode: ", type(arg), arg)
        return "a simple string object"

In [4]:
json.loads(j, cls=CustomDecoder)

decode:  <class 'str'> 
{
    "a": 100,
    "b": [1,2,3],
    "c": "python",
    "d": {
    "e": 4,
    "f": 5.5
    }
}


'a simple string object'

In [5]:
class Point:
    def __init__(self, x ,y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [8]:
j_points = '''
{
    "points": [
        [10,20],
        [-1, -2],
        [0.5, 0.5]
    ]
}'''

j_other = '''
{
    "a": 1,
    "b": 2
}
'''

In [9]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        if 'points' in arg:
            obj = json.loads(arg)
            return "parsing object for points"
        else:
            return super().decode(arg)

In [10]:
json.loads(j_points, cls=CustomDecoder)

'parsing object for points'

In [11]:
json.loads(j_other, cls=CustomDecoder)

{'a': 1, 'b': 2}

In [14]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        if 'points' in obj:
            obj['points'] = [Point(x,y) 
                             for x,y in obj['points']]
        return obj

In [15]:
json.loads(j_points, cls=CustomDecoder)

{'points': [Point(x=10, y=20), Point(x=-1, y=-2), Point(x=0.5, y=0.5)]}

In [16]:
json.loads(j_other, cls=CustomDecoder)

{'a': 1, 'b': 2}

In [17]:
json.loads(j, cls=CustomDecoder)

{'a': 100, 'b': [1, 2, 3], 'c': 'python', 'd': {'e': 4, 'f': 5.5}}

In [18]:
import re

In [19]:
pattern = r'"_type"\s*:\s*"point"'

In [20]:
print(r'word\tword')

word\tword


In [21]:
regexp = re.compile(pattern)

In [22]:
print(regexp.search('"a": 1'))

None


In [25]:
print(regexp.search('    "_type"    :\t "point"'))

<re.Match object; span=(4, 25), match='"_type"    :\t "point"'>


In [27]:
re.search(pattern, '    "_type"    :\t "point"')

<re.Match object; span=(4, 25), match='"_type"    :\t "point"'>

In [28]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get('_type', None) == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj

In [29]:
j = '''
{
    "a": 100,
    "b": 0.5,
    "rectangle": {
        "corners": {
            "b_left": {"_type": "point", "x": -1, "y": -1},
            "b_right": {"_type": "point", "x": 1, "y": -1},
            "t_left": {"_type": "point", "x": -1, "y": 1},
            "t_right": {"_type": "point", "x": 1, "y": 1}
        },
        "rotate": {"_type" : "point", "x": 0, "y": 0},
        "interior_pts": [
            {"_type": "point", "x": 0, "y": 0},
            {"_type": "point", "x": 0.5, "y": 0.5}
        ]
    }
}
'''

In [30]:
json.loads(j)

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
   'b_right': {'_type': 'point', 'x': 1, 'y': -1},
   't_left': {'_type': 'point', 'x': -1, 'y': 1},
   't_right': {'_type': 'point', 'x': 1, 'y': 1}},
  'rotate': {'_type': 'point', 'x': 0, 'y': 0},
  'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
   {'_type': 'point', 'x': 0.5, 'y': 0.5}]}}

In [31]:
from pprint import pprint
pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [32]:
from decimal import Decimal

In [33]:
CustomDecoder = json.JSONDecoder(parse_float=Decimal)

In [34]:
d = CustomDecoder.decode(j)

In [35]:
d

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
   'b_right': {'_type': 'point', 'x': 1, 'y': -1},
   't_left': {'_type': 'point', 'x': -1, 'y': 1},
   't_right': {'_type': 'point', 'x': 1, 'y': 1}},
  'rotate': {'_type': 'point', 'x': 0, 'y': 0},
  'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
   {'_type': 'point', 'x': Decimal('0.5'), 'y': Decimal('0.5')}]}}

In [36]:
pprint(d)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
                           'b_right': {'_type': 'point', 'x': 1, 'y': -1},
                           't_left': {'_type': 'point', 'x': -1, 'y': 1},
                           't_right': {'_type': 'point', 'x': 1, 'y': 1}},
               'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
                                {'_type': 'point',
                                 'x': Decimal('0.5'),
                                 'y': Decimal('0.5')}],
               'rotate': {'_type': 'point', 'x': 0, 'y': 0}}}


In [39]:
class CustomDecoder(json.JSONDecoder):
    base_decoder = json.JSONDecoder(parse_float=Decimal)

    def decode(self, arg):
        obj = self.base_decoder.decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj

    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get('_type', None) == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj

In [40]:
pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [41]:
result = json.loads(j, cls=CustomDecoder)

In [42]:
pt = result['rectangle']['interior_pts'][1]

In [43]:
type(pt.x)

decimal.Decimal

In [44]:
class CustomDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(parse_float=Decimal)

    def decode(self, arg):
        obj = super().decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj

    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get('_type', None) == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj

In [45]:
result = json.loads(j, cls=CustomDecoder)

In [46]:
pprint(result)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [47]:
pt = result['rectangle']['interior_pts'][1]

In [48]:
print(type(pt.y))

<class 'decimal.Decimal'>
